<a href="https://colab.research.google.com/github/sturu1/git-first/blob/master/Copy_of_sung_lec02_multilinear_regression(3)_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 정규화

> 본 실습에서는 데이터 정규화 (normalization)에 대해서 살펴보겠습니다. 

> 딥러닝을 포함한 대부분의 머신러닝에서는 입력데이터의 정규화가 필요합니다. 입력변수들이 비슷한 값의 범위를 갖는다면 정규화가 필요 없을 수도 있지만, 변수별로 값의 범위(scale)이 너무 상이하면 학습이 잘 진행되지 않기 때문입니다. 

> 특히, 딥러닝에서는 입력변수의 값을 [0,1] 사이의 값으로 바꿔주는 min-max 정규화나 통계에서 z분포를 사용하는 표준화를 많이 사용합니다. 우리는 이번 예제를 통해서 min-max 정규화에 대해 살펴보겠습니다. 

> 먼저, 정규화를 진행하지 않고, 다음과 같은 데이터에 대한 학습을 진행해 보겠습니다.

In [0]:
import numpy as np

In [0]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

In [4]:
xy.shape

(8, 5)

> 데이터를 살펴보면, 3번째 열의 값이 다른 열의 값보다 너무 큰 값을 보임을 알 수 있습니다(약 2000배). 이렇듯 각 변수의 값의 범위가 큰 차이가 날때, 학습이 어떻게 진행되는지 확인해 보겠습니다. 

> 먼저 데이터를 입력 및 출력 데이터로 분활합니다. 

In [0]:
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

In [6]:
x_data.shape, y_data.shape

((8, 4), (8, 1))

필요한 모듈을 불러와, 모델을 생성하겠습니다. 

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
model1 = Sequential()
model1.add(Dense(units=1, input_shape=(4, )))
model1.compile(optimizer=SGD(lr=0.01), loss='mse')

In [26]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


1000번의 epoch을 거쳐 학습을 진행해 보겠습니다. 

In [0]:
history1 = model1.fit(x_data, y_data, epochs=1000)

> 학습이 진행되지 않아 손실함수의 값이 `nan` 값이 나오는 것을 알 수 있습니다. 

> 다음은 각 입력변수의 값의 범위를 [0,1] 사이로 변환해주는 정규화 작업을 수행하겠습니다. 이때 머신러닝에서 많이 사용하는 skitlearn 모듈의 MinMaxScaler라는 클래스를 사용하겠습니다. 

In [0]:
from sklearn.preprocessing import MinMaxScaler

> `MinMaxScaler` 클래스를 이용하여 `scaler`라는 instance를 생성하였습니다. 이 instance를 통하여 데이터를 [0,1] 사이의 값으로 정규화 시킬 수 있습니다. 

In [0]:
scaler=MinMaxScaler(feature_range=(0,1))

> 이제 `scaler` 클래스의 맴버함수인 `fit_transform` 함수를 이용하여 우리의 입력데이터를 정규화 시켜 보겠습니다. 

In [0]:
x_data = scaler.fit_transform(x_data)

> 이제 우리의 입력데이터를 확인해 볼까요? 

In [10]:
x_data

array([[1.        , 1.        , 0.        , 1.        ],
       [0.70548491, 0.70439552, 1.        , 0.71881783],
       [0.54412549, 0.50274824, 0.57608696, 0.60646801],
       [0.33890353, 0.31368023, 0.10869565, 0.45989134],
       [0.51436   , 0.4258239 , 0.30434783, 0.58504805],
       [0.49556179, 0.4258239 , 0.31521739, 0.48131134],
       [0.11436064, 0.        , 0.20652174, 0.22007776],
       [0.        , 0.07747099, 0.5326087 , 0.        ]])

> 각 변수(속성)별로 최소 0 에서 최대 1까지의 값을 갖는 것을 알 수 있습니다. 

> 다음으로 변환된 입력변수를 기반으로한 새로운 모델을 생성해 보겠습니다. 

In [11]:
model2 = Sequential()
model2.add(Dense(units=1, input_shape=(4, )))
model2.compile(optimizer=SGD(lr=0.01), loss='mse')
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


> 학습이 잘 진행되는 것을 알 수 있습니다. 

> 학습이 끝난 이 모델이(model_2) x_data에 대해 어떤 출력값을 예측해 냈는지 확인해 보겠습니다. 

In [0]:
history2 = model2.fit(x_data, y_data, epochs=1000)

In [49]:
model2.predict(x_data)

array([[823.57965],
       [841.07947],
       [829.20483],
       [813.3823 ],
       [823.2528 ],
       [816.5451 ],
       [810.5697 ],
       [801.5197 ]], dtype=float32)

In [12]:
x_data

array([[1.        , 1.        , 0.        , 1.        ],
       [0.70548491, 0.70439552, 1.        , 0.71881783],
       [0.54412549, 0.50274824, 0.57608696, 0.60646801],
       [0.33890353, 0.31368023, 0.10869565, 0.45989134],
       [0.51436   , 0.4258239 , 0.30434783, 0.58504805],
       [0.49556179, 0.4258239 , 0.31521739, 0.48131134],
       [0.11436064, 0.        , 0.20652174, 0.22007776],
       [0.        , 0.07747099, 0.5326087 , 0.        ]])

In [39]:
y_data

array([[831.659973],
       [828.070007],
       [824.159973],
       [819.23999 ],
       [818.97998 ],
       [820.450012],
       [813.669983],
       [809.559998]])

참고로 추가적인 입력데이터에 대해 모델의 성능을 테스트해보고자 한다면, 
추가 입력데이터를 scaler instance를 이용하여 [0,1]사이의 값으로 변환해 주어야 합니다. 예를 들어 입력 데이터가 `[810, 830, 100000,783]` 인 경우 다음과 같이 하면 됩니다. 

In [14]:
model2.predict(scaler.transform([[810, 830, 100000, 783]]))

array([[-0.42613676]], dtype=float32)